# PySurfer for EEG results visualization in source space

## Goal

In this notebook we project statistical results calculated on Brainstorm source-space parcellations to the 'fsaverage' surface. We assume that the DKT atlas (31 areas / hemisphere) was selected for source-space ROIs.



## Prerequisites

(1) .annot file for the parcellation. Ours is from the Mindboggle dataset (https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/XCCE9Q). Specifically, we applied the lh.DKT31atlas101subjects.gcs / rh.DKT31atlas101subjects.gcs atlases to the fsaverage subject using mris_ca_label (Freesurfer tool): <br>

<code> mris_ca_label fsaverage rh $SUBJECTS_DIR/fsaverage/surf/rh.sphere.reg "atlas dir"/rh.DKT31atlas101subjects.gcs "output dir"/"output file name".annot </code> 
<br>

(2) PySurfer. Watch out for a nasty rendering bug appearing for certain Linux configs, description and solution here: https://github.com/nipy/PySurfer/pull/286
<br>

(3) Results / stats array, one value per ROI/area. We use .csv files where the first column stores ROI names/labels.
<br>

That's all, let's get into it!
<br>


## Imports

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mayavi import mlab
from tvtk.api import tvtk
from tvtk.common import configure_input_data
from surfer import Brain
import nibabel as nib

## Load parcellation and results to visualize

In [2]:
# Load annotations

# freesurfer parcellation files (*.annot), one per hemisphere
aparc_file_lh = '/home/adamb/Downloads/freesurfer_atlas_DKT31labels101subjects/lh_test_fsaverage.annot'
aparc_file_rh = '/home/adamb/Downloads/freesurfer_atlas_DKT31labels101subjects/rh_test_fsaverage.annot'

# nibabel loads *.annot files out-of-the-box
vtx_labels_lh, ctab_lh, aparc_names_lh = nib.freesurfer.read_annot(aparc_file_lh)
vtx_labels_rh, ctab_rh, aparc_names_rh = nib.freesurfer.read_annot(aparc_file_rh)

# turn roi names to string from byte text
aparc_names_lh = [name.decode('utf-8') for name in aparc_names_lh]
aparc_names_rh = [name.decode('utf-8') for name in aparc_names_rh]


# Load results to map

# frequency band of interest
freq = 'alpha'

# *.csv with results we want to visualize, one per hemisphere
res_file_lh = '/media/adamb/bonczData/hyperscan/circCorr_fixM_leftHemi_' + freq +'.csv'
res_file_rh = '/media/adamb/bonczData/hyperscan/circCorr_fixM_rightHemi_' + freq +'.csv'

# load *.csv files into numpy arrays
res_lh = np.genfromtxt(res_file_lh, delimiter=',', dtype=None, encoding='utf8')
res_rh = np.genfromtxt(res_file_rh, delimiter=',', dtype=None, encoding='utf8')

# roi names to a list, stats to array, skip first value (header)
res_names_lh = res_lh[1:,0].tolist()
res_names_rh = res_rh[1:,0].tolist()
res_values_lh = res_lh[1:, 3]
res_values_rh = res_rh[1:, 3]
res_values_lh = res_values_lh.astype('float64')  # specify data type as double float
res_values_rh = res_values_rh.astype('float64')

## Optional: match results to annotation labels

Our ROI labels do not match exactly the ones from the annotation files. First, there is a cardinality-discrepancy, as the DTK annotation retains all labels from the DK atlas without using them all (does not assign vertices to some of them). Second, our results files contain labels with an "L" or "R" tag.
<br> <br>
Let's make sure first that there are indeed ROI names included in the annotations without any corresponding vertices:
<br> <br>

In [3]:
# delete the trailing "L" / "R" at the end of each result roi label
res_names_lh = [label[0:-2] for label in res_names_lh]
res_names_rh = [label[0:-2] for label in res_names_rh]

# check the range of values in labels_lh, compare them to names_lh
print('Unique values from variable \"vtx_labels_lh\":')
print(np.unique(vtx_labels_lh))
print('No. of unique values:', len(np.unique(vtx_labels_lh)))
# There are more names than unique labels:
print('There are ', len(aparc_names_lh), ' ROI names in \"aparc_names_lh\" though')

# same for right hemi
print('\nUnique values from variable \"vtx_labels_rh\":')
print(np.unique(vtx_labels_rh))
print('No. of unique values:', len(np.unique(vtx_labels_rh)))
print('There are ', len(aparc_names_rh), ' ROI names in \"aparc_names_rh\" though')

# ROIs with no vertices
print('\nThe following ROIs have no vertices assigned to them:')
print('Left hemi:')
print([aparc_names_lh[i] for i in np.setdiff1d(np.arange(len(aparc_names_rh)), np.unique(vtx_labels_lh))])
print('Right hemi:')
print([aparc_names_lh[i] for i in np.setdiff1d(np.arange(len(aparc_names_rh)), np.unique(vtx_labels_rh))])


Unique values from variable "vtx_labels_lh":
[ 0  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 34 35]
No. of unique values: 32
There are  36  ROI names in "aparc_names_lh" though

Unique values from variable "vtx_labels_rh":
[ 0  2  3  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 34 35]
No. of unique values: 32
There are  36  ROI names in "aparc_names_rh" though

The following ROIs have no vertices assigned to them:
Left hemi:
['bankssts', 'corpuscallosum', 'frontalpole', 'temporalpole']
Right hemi:
['bankssts', 'corpuscallosum', 'frontalpole', 'temporalpole']


Hopefully the above lists consist of (1) Banks of the sup. temporal sulcus; (2) Corpus callosum; (3) Frontal pole; (4) Temporal pole. <br> <br>
(See the background for this whole issue around ROIs included/excluded here: https://mindboggle.readthedocs.io/en/latest/labels.html)
<br> <br>
If yes, now we can make sure our labels match those used for ROIs with vertices. We take the set differences and expect to see the four ROIs specified above + the "unknown" category
<br>

In [4]:
# check if our roi names set covers the annotation roi names set after deleting the ones without vertices
print(set(aparc_names_lh) - set(res_names_lh))
print(set(aparc_names_rh) - set(res_names_rh))

{'unknown', 'frontalpole', 'temporalpole', 'bankssts', 'corpuscallosum'}
{'unknown', 'frontalpole', 'temporalpole', 'bankssts', 'corpuscallosum'}


Great, now we can define the mapping from result ROI labels to annotation ROI labels - the same mapping can then be used for defining the colors
<br> <br>

In [5]:
res2aparc_mapping_lh = [aparc_names_lh.index(res_names_lh[i]) for i in range(len(res_names_lh))]
res2aparc_mapping_rh = [aparc_names_rh.index(res_names_rh[i]) for i in range(len(res_names_rh))]

# sanity check - are these two mappings the same?
if res2aparc_mapping_lh == res2aparc_mapping_rh:
    print('Sanity check passed: Mappings for the two hemispheres are the same, use the \"res2aparc\" list')
    res2aparc = res2aparc_mapping_lh
    
# sanity check - applying the mapping to ROI names in res_names_lh / -rh should 
# give back aparc_names_lh / -rh but without the ROIs with no vertices assigned
res_names_reordered_lh = [None]*len(aparc_names_lh)  # preallocate the ROI name list
for pos in range(len(res_names_lh)):
    res_names_reordered_lh[res2aparc[pos]] = res_names_lh[pos]  
# pritn the two lists side-by-side
print('\n\nROI names list from results file after reordering,',
      'compared to parcellation ROI names list (side-by-side):\n')
fmt = '{:<8}{:<28}{}'
print(fmt.format('', 'Aparc_names list', 'Reordered res_names list'))
for i, (aparc_name, res_name_ro) in enumerate(zip(aparc_names_lh, res_names_reordered_lh)):
    print(fmt.format(i, aparc_name, res_name_ro))

Sanity check passed: Mappings for the two hemispheres are the same, use the "res2aparc" list


ROI names list from results file after reordering, compared to parcellation ROI names list (side-by-side):

        Aparc_names list            Reordered res_names list
0       unknown                     None
1       bankssts                    None
2       caudalanteriorcingulate     caudalanteriorcingulate
3       caudalmiddlefrontal         caudalmiddlefrontal
4       corpuscallosum              None
5       cuneus                      cuneus
6       entorhinal                  entorhinal
7       fusiform                    fusiform
8       inferiorparietal            inferiorparietal
9       inferiortemporal            inferiortemporal
10      isthmuscingulate            isthmuscingulate
11      lateraloccipital            lateraloccipital
12      lateralorbitofrontal        lateralorbitofrontal
13      lingual                     lingual
14      medialorbitofrontal         medialorbitof

Now we can finally order our results according to the parcellation name list, which also gives the position in the colormap.
<br> <br> 

In [6]:
# preallocate
res_values_reordered_lh = np.zeros((len(aparc_names_lh),))
res_values_reordered_rh = np.zeros((len(aparc_names_rh),))
# reorder values
for pos in range(len(res_names_lh)):
    res_values_reordered_lh[res2aparc[pos]] = res_values_lh[pos]
    res_values_reordered_rh[res2aparc[pos]] = res_values_rh[pos]
# switch all zeros to nan
res_values_reordered_lh[np.isnan(res_values_reordered_lh)] = 0
res_values_reordered_rh[np.isnan(res_values_reordered_rh)] = 0

## Assign stat values to vertices, based on ROI membership

The \"vtx_labels_lh\" variables from the annotation file hold the ROI membership of each vertex. We simply create an array that switches ROI memberships to stat values:
<br> <br>


In [7]:
vtx_res_values_lh = res_values_reordered_lh[vtx_labels_lh]
vtx_res_values_rh = res_values_reordered_lh[vtx_labels_rh]

## Plot brain surface and overlay ROI values

We are finally ready to plot the left-right surfaces and overlay the stat values on them.
<br> <br>

In [44]:
# enable IPython GUI event loop integration
%gui qt

# define surface plotting details
surf_value = 'pial'
hemi_value = 'lh'
views_list = ['lat', 'ven', 'med']
fig_size = (600, 1000)
hemi_text = 'L'
filename = 'resTest'
outdir = '/home/adamb/jupyter_notebooks'  # target dir for saving out images

# basic pysurfer init command
brain = Brain('fsaverage', 
              hemi=hemi_value, 
              surf=surf_value,
              views=views_list,
              show_toolbar=False,
              size=fig_size,
              background='black')

# add a "left" / "right" label to the figure
brain.add_text(x=0.1,
              y=0.8,
              text=hemi_text,
              name='hemi_label',
              font_size=20)


In [43]:
# stat value overlay details
min_value = 0.01
max_value = np.nanmax(res_values_lh)
color_map = 'autumn'
thresh_value = np.nanmin(res_values_lh)

# overlay stat values as assigned to vertices
brain.add_data(vtx_res_values_lh, 
               min=min_value,
               max=max_value, 
               colormap=color_map, 
               thresh=thresh_value, 
               hemi=hemi, 
               transparent=True)

colormap sequential: [1.00e-02, 8.38e-01, 1.67e+00] (transparent)
